# GWAS interpretation
Here we're going to explore and interprete the results obtained from GWAS.

## Manhattan plot

## Quantile-quantile plot

## Population structure

## Give me a peak
please change title

## Multiple testing

## Minor allele frequency (MAF)

## Nice peak ... now what?